In [90]:
import re
from enum import Enum
from operator import attrgetter

'''
Use in ExtractStationLine class as parameters
'''
class StationLine(Enum):
    red = 1
    green = 2
    circle = 3
    purple = 4
    blue = 5
    brown = 6
    none = 99

# model object to hold name and score
class StationModel:
    stationName = ""
    score = 0
    
    def __init__(self, mStationName, mScore=0):
        self.stationName = mStationName
        self.score = mScore
    
    def getName(self):
        return self.stationName
    
    def getScore(self):
        return self.score
    
    def addScore(self, mScore):
        self.score += mScore
        
    def getString(self):
        return "{}: {}".format(self.getName(), self.getScore())

class SSPairArr:
    ssPairArr = []
    
    def __init__(self, lineArr):        
        for line in lineArr:
            self.ssPairArr.append(StationModel(line, 0))
        
    # find station by name
    def get(self, stationName):
        for pair in self.ssPairArr:
            if pair.getName().lower() == stationName.lower():
                return pair
        return None
    
    # return the max value in models, can contain multiple same max value
    def getMax(self):
        return max(self.ssPairArr, key=attrgetter('score')).getScore()
    
    # sanity check on the models
    def mPrint(self):        
        for model in self.ssPairArr:
            print(model.getString())

    
'''
Extraction station line in sentence
'''
class ExtractStationLine:
    redLineList = ["[nsl]","[nsl]:", "ns", "nsl", "north south", "north south line", "nsline", "ns line"]
    greenLineList = ["[ewl]","[ewl]:","ew", "ewl", "east west", "east west line", "ewline", "ew line"]
    circleLineList = ["[ccl]","[ccl]:","cc", "ccl", "circle line", "circleline", "cc line"]
    purpleLineList = ["[nel]","[nel]:","ne", "nel", "north east", "north east line", "neline", "ne line"]
    blueLineList = ["[dtl]","[dtl]:","dt", "dtl", "down town", "down town line", "dtline", "dt line"]
    brownLineList = ["[tel]","[tel]:","te", "tel", "thomson-east Coast", "thomson-east Coast line", "tecline", "tec line"]    
    
    # check group of stations belongs to which line
    def checkStationJson(self, jsonData):                
        lineArr = ["NS", "EW", "CC", "NE", "DT", "TE"]
        ssPairArr = SSPairArr(lineArr)    
        
        for data in jsonData:
            for line in lineArr:
                if re.search(line, data["id"], re.IGNORECASE):
                    pair = ssPairArr.get(line)
                    pair.addScore(1)                        
        ssPairArr.mPrint()
        print(ssPairArr.getMax())

        
#         for data in jsonData:        
#             if re.search("ns", data["id"], re.IGNORECASE):
#                 ssPairArr.getScore += 1
#             if re.search("ew", data["id"], re.IGNORECASE):
#                 ssPairArr.getScore += 1
#             if re.search("cc", data["id"], re.IGNORECASE):
#                 ssPairArr.getScore += 1
#             if re.search("ne", data["id"], re.IGNORECASE):
#                 ssPairArr.getScore += 1
#             if re.search("dt", data["id"], re.IGNORECASE):
#                 ssPairArr.getScore += 1
#             if re.search("te", data["id"], re.IGNORECASE):
#                 ssPairArr.getScore += 1
                
#         highest = max(scoreArr)
#         # check for multiple high score
#         highScoreCount = 0
#         firstHighIndex = 0
#         for idx, score in enumerate(scoreArr):
#             if score == highest:
#                 if highScoreCount == 0:
#                     firstHighIndex = idx # grab the first highscore
#                 highScoreCount += 1
#         # if only 1 high score return result, else compare stations and return shortest line
#         if(highScoreCount == 1):
#             return lineArr[firstHighIndex]
#         else:
#             stations = data["id"].split(";")
            
    
    def checkStationLine(self, jsonData):
        if self.containKey(StationLine.red, jsonData):
            return "NS"
        elif self.containKey(StationLine.green, jsonData):
            return "EW"
        elif self.containKey(StationLine.circle, jsonData):
            return "CC"
        elif self.containKey(StationLine.purple, jsonData):
            return "NE"
        elif self.containKey(StationLine.blue, jsonData):
            return "DT"
        elif self.containKey(StationLine.brown, jsonData):
            return "TE"
        else:
            return "NONE"
            
    def containKey(self, stationType, jsonData):
        if stationType is StationLine.red:
            lineList = self.redLineList
        elif stationType is StationLine.green:
            lineList = self.greenLineList
        elif stationType is StationLine.circle:
            lineList = self.circleLineList
        elif stationType is StationLine.purple:
            lineList = self.purpleLineList
        elif stationType is StationLine.blue:
            lineList = self.blueLineList
        elif stationType is StationLine.brown:
            lineList = self.brownLineList
#         print(lineList)
        
        isContainKey = False
        for key in lineList:    
            result = self.findWholeWord(key)(jsonData)
            if result is not None:        
                isContainKey = True
                break
        return isContainKey

    def findWholeWord(self, w):
        return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

In [2]:
import import_notebook
from readWrite import ReadWrite
import re

class ExtractStation:
    stationFile = "station_translation.json"
    abbFile = "abb_replacement.json"
    stationJson = ""
    abb_replace = ""
    
    def __init__(self):
        readWrite = ReadWrite()
        self.getStationsJson()
        self.abb_replace = readWrite.readJsonFile(self.abbFile)
    
    # Return list of station in json format
    def getStationsJson(self):
        readWrite = ReadWrite()
        self.stationJson = readWrite.readJsonFile(self.stationFile)
        print("Total stations: %d" % len(self.stationJson))
        print("Keys: ch_name, id, name")
        return self.stationJson
    
    # Extract sentence is reference to which station and which line
    def extract(self, sentence):
        stationObjArr = []
        matchObjArr = []
        sentence = self.replaceAbbsWords(sentence)      
        for json in self.stationJson:
            name = json['name']                                      
            isFound, matchObj = self.containWord(name, sentence)
            if(isFound):                
                stationObjArr.append(json) 
                matchObjArr.append(matchObj)
        # preserve stations direction in sentence        
        stationObjArr = self.sortByMatchObj(matchObjArr, stationObjArr)                      
        return stationObjArr

    # Check if key word is found in sentence
    def containWord(self, key, sentence):                
        isContainKey = False        
        result = self.findWholeWord(key)(sentence)        
        if result is not None:        
            isContainKey = True
        return isContainKey, result
        
    def findWholeWord(self, w):
        return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search      
    
    # sorting of matchObj using span, return sorted stationObj
    def sortByMatchObj(self, matchObjArr, stationObjArr):
        indexList = []
        for matchObj in matchObjArr:
            indexList.append(matchObj.span(1)[0])
        indexList = sorted(indexList)
        result = []
        for index in indexList:
            for idx, matchObj in enumerate(matchObjArr):
                if matchObj.span(1)[0] == index:
                    result.append(stationObjArr[idx])
        return result
                            
    def replaceAbbsWords(self, sentence): 
        try:
            for key in self.abb_replace:        
                sentence = re.sub(r"\b{}\b".format(key), self.abb_replace[key], sentence, flags=re.IGNORECASE)    
                sentence = sentence.replace('&amp;', 'and')
            return sentence
        except Exception as e:
            print("replaceAbbsWords Error: {}".format(e))

importing Jupyter notebook from readWrite.ipynb


In [91]:
# Testing - identifying by type of line as keywords
extStLine = ExtractStationLine()
result = extStLine.checkStationLine("This sentence contain key words for ewl some rubbish words train station line")
print(result)

# Testing - identifying station names as keywords
extStation = ExtractStation()
stationJson = extStation.extract("[nel] This sentence contain serangoon, dg in it and it should return the correct json object")
for item in stationJson:
    print(item)

# Testing - checking which group of stations belongs to which line
extStLine.checkStationJson(stationJson)

EW
Total stations: 140
Keys: ch_name, id, name
{'name': 'Serangoon', 'id': 'NE12;CC13', 'ch_name': '实龙岗'}
{'name': 'Dhoby Ghaut', 'id': 'NE6;NS24;CC1', 'ch_name': '多美歌'}
NS: 1
EW: 0
CC: 2
NE: 2
DT: 0
TE: 0
2
